In [ ]:
from nids_datasets import Dataset
import pandas as pd
from pyarrow.parquet import ParquetFile
import pyarrow as pa

### 1. Pobranie danych

In [ ]:
data = Dataset(dataset='CIC-IDS2017', subset=['Packet-Fields'], files='all')
df_info = DatasetInfo(dataset='CIC-IDS2017')
print(df_info)

       BENIGN  FTP-Patator  SSH-Patator  DoS slowloris  DoS Slowhttptest  \
File                                                                       
1     4822905            0            0              0                 0   
2     3571097            0            0              0                 0   
3     3556526         1953            0              0                 0   
4     2575904        12803            0              0                 0   
5     3007086        68205            0              0                 0   
6     2637797        28602        38680              0                 0   
7     2612615            0       124631              0                 0   
8     2649076            0            0              0                 0   
9     2948428            0            0          47519             19866   
10     422225            0            0              0             19675   
11    4418585            0            0             41                 0   
12    265138

#### UWAGA - Cały dataset jest bardzo duży, więc pobieranie może zająć dużo czasu i miejsca na dysku (ok. 250GB).

In [ ]:
# UWAGA - Cały dataset jest bardzo duży, więc pobieranie może zająć dużo czasu i miejsca na dysku (ok. 250GB).
# data.download()

### 2. Przetowrzenie danych z foramtu Parquet do formatu CSV

In [ ]:
# Load the first file to get column names
pf = ParquetFile('CIC-IDS2017/Packet-Fields/Packet_Fields_File_1.parquet')

if pf.num_row_groups > 0:
    # Read just one row to get column structure
    first_batch = next(pf.iter_batches(batch_size=1))
    df = pa.Table.from_batches([first_batch]).to_pandas()
    
    # Get column names
    columns = list(df.columns)
    
    # Save to text file
    with open('cicids2017_columns.txt', 'w') as f:
        for col in columns:
            f.write(col + '\n')
    
    print(f"Saved {len(columns)} column names to 'cicids2017_columns.txt'")
    print(f"First few columns: {columns[:5]}")

else:
    print("File has no data")


Saved 261 column names to 'cicids2017_columns.txt'
First few columns: ['packet_id', 'flow_id', 'source_ip', 'source_port', 'destination_ip']


In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import os
import zipfile
from pathlib import Path

# Kolumny które potrzebujesz
needed_columns = [
    "Ethernet type",
    "IP proto", 
    "IP flags",
    "TCP dport",
    "UDP dport", 
    "IP len",
    "TCP flags",
    "attack_label",
    "flow_id"
]   

# Ścieżka do plików Parquet
parquet_directory = "giga_miejsca/CIC-IDS2017/Packet-Fields"

# Utwórz katalog dla plików CSV
csv_directory = "giga_miejsca/converted_csv_files"
Path(csv_directory).mkdir(exist_ok=True)

# Przetwarzaj wszystkie 18 plików
for file_num in range(1, 19):  # Pliki od 1 do 18
    input_file = os.path.join(parquet_directory, f'Packet_Fields_File_{file_num}.parquet')
    output_file = os.path.join(csv_directory, f'Packet_Fields_File_{file_num}.csv')
    
    # Sprawdź czy plik istnieje
    if not os.path.exists(input_file):
        print(f"Plik {input_file} nie istnieje, pomijam...")
        continue
    
    print(f"Przetwarzanie pliku: {input_file}")
    
    # Otwórz plik Parquet
    parquet_file = pq.ParquetFile(input_file)
    
    # Przetwarzaj w batch'ach
    batch_size = 10000  # Dostosuj rozmiar batch'a do swojej pamięci
    
    for i, batch in enumerate(parquet_file.iter_batches(
        batch_size=batch_size, 
        columns=needed_columns
    )):
        # Konwertuj batch do pandas DataFrame
        df_chunk = batch.to_pandas()
        
        # Ustaw tryb zapisu
        mode = 'w' if i == 0 else 'a'
        header = i == 0  # Nagłówek tylko dla pierwszego batch'a
        
        # Zapisz do CSV
        df_chunk.to_csv(
            output_file,
            mode=mode,
            header=header,
            index=False
        )
        
        if i == 0:  # Informacja tylko dla pierwszego batcha każdego pliku
            print(f"  Rozpoczęto konwersję, batch size: {batch_size}")
    
    print(f"  Zakończono konwersję: {input_file} -> {output_file}")

print("Wszystkie pliki zostały przetworzone!")

# # Utwórz archiwum ZIP z plikami CSV
# zip_filename = "packet_fields_csv.zip"
# print(f"\nTworzenie archiwum ZIP: {zip_filename}")

# with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
#     # Dodaj wszystkie pliki CSV do archiwum
#     for csv_file in os.listdir(csv_directory):
#         if csv_file.endswith('.csv'):
#             file_path = os.path.join(csv_directory, csv_file)
#             # Dodaj plik do ZIP (bez ścieżki katalogu)
#             zipf.write(file_path, csv_file)
#             print(f"  Dodano do archiwum: {csv_file}")

# print(f"\nArchiwum ZIP zostało utworzone: {zip_filename}")
# print(f"Pliki CSV znajdują się w katalogu: {csv_directory}")


Przetwarzanie pliku: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_1.parquet
  Rozpoczęto konwersję, batch size: 10000
  Zakończono konwersję: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_1.parquet -> giga_miejsca/converted_csv_files/Packet_Fields_File_1.csv
Przetwarzanie pliku: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_2.parquet
  Rozpoczęto konwersję, batch size: 10000
  Zakończono konwersję: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_2.parquet -> giga_miejsca/converted_csv_files/Packet_Fields_File_2.csv
Przetwarzanie pliku: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_3.parquet
  Rozpoczęto konwersję, batch size: 10000
  Zakończono konwersję: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_3.parquet -> giga_miejsca/converted_csv_files/Packet_Fields_File_3.csv
Przetwarzanie pliku: giga_miejsca/CIC-IDS2017/Packet-Fields/Packet_Fields_File_4.parquet
  Rozpoczęto konwersję, batch size: 10000
  Zakończono konw

In [ ]:
import pandas as pd
import os

# Directory containing the CSV files
directory = 'packet_fields_csv'

# List to hold filtered dataframes
filtered_dfs = []

# Loop through files Packet_Fields_File_X.csv where X is 1 to 18
for i in range(1, 19):
    file_name = f'Packet_Fields_File_{i}.csv'
    file_path = os.path.join(directory, file_name)
    
    if os.path.exists(file_path):
        try:
            # Read individual file
            df = pd.read_csv(file_path)
            print(f"Reading {file_name} with {len(df)} rows")
            
            # Filter to one random packet per flow_id for this file
            filtered_df = df.groupby('flow_id').apply(lambda x: x.sample(1)).reset_index(drop=True)
            
            # Add to list
            filtered_dfs.append(filtered_df)
            
            print(f"  -> Filtered to {len(filtered_df)} unique flow_ids")
            
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    else:
        print(f"File not found: {file_name}")

# Combine all filtered dataframes
if filtered_dfs:
    final_df = pd.concat(filtered_dfs, ignore_index=True)
    
    # Save the final dataset
    output_file = 'filtered_packet_fields.csv'
    final_df.to_csv(output_file, index=False)
    
    print(f"\nFinal dataset saved as '{output_file}'")
    print(f"Total rows in final dataset: {len(final_df)}")
    print(f"Files processed: {len(filtered_dfs)}")
    print(f"Columns: {list(final_df.columns)}")
    
else:
    print("No files were successfully processed.")

zip_filename = "filtered_packet_fields.zip"
print(f"\nTworzenie archiwum ZIP: {zip_filename}")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    file_path = os.path.join(csv_directory, output_file)
    zipf.write(file_path, output_file)
    print(f"  Dodano do archiwum: {output_file}")
           
print(f"\nArchiwum ZIP zostało utworzone: {zip_filename}")